In [ ]:
# default_exp models.TabModel

# TabModel

> This is an implementation created by Ignacio Oguiza based on fastai's TabularModel - oguiza@gmail.com. I build it so that it's easy to change the head of the model, something that is particularly interesting when building hybrid models.

In [ ]:
#export
from tsai.imports import *
from tsai.models.layers import *

In [ ]:
from tsai.data.tabular import *
from tsai.models.utils import *

In [ ]:
#export
class TabModel(Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs, n_cont, out_sz, layers, ps=None, embed_p=0., y_range=None, use_bn=True, bn_final=False, bn_cont=True, 
                 act=nn.ReLU(inplace=True)):
        ps = ifnone(ps, [0.]*len(layers))
        if not is_listy(ps): ps = [ps]*len(layers)
        self.embeds = nn.ModuleList([Embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(embed_p)
        self.bn_cont = nn.BatchNorm1d(n_cont) if bn_cont else None
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont = n_emb,n_cont
        sizes = [n_emb + n_cont] + layers + [out_sz]
        actns = [act for _ in range(len(sizes)-2)]
        _layers = [LinBnDrop(sizes[i], sizes[i+1], bn=use_bn and (i!=len(actns)-1 or bn_final), p=p, act=a)
                       for i,(p,a) in enumerate(zip(ps,actns))]
        self.layers = nn.Sequential(*_layers)
        _head = [nn.Linear(layers[-1], out_sz)]
        self.head_nf = layers[-1]
        if y_range is not None: _head.append(SigmoidRange(*y_range))
        self.head = nn.Sequential(*_head)

    def forward(self, x_cat, x_cont=None):
        x = self._backbone_forward(x_cat, x_cont)
        return self.head(x)
    
    def _backbone_forward(self, x_cat, x_cont=None):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            if self.bn_cont is not None: x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        x = self.layers(x)
        return x

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
# df['salary'] = np.random.rand(len(df)) # uncomment to simulate a cont dependent variable
procs = [Categorify, FillMissing, Normalize]
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
y_names = ['salary']
y_block = RegressionBlock() if isinstance(df['salary'].values[0], float) else CategoryBlock()
splits = RandomSplitter()(range_of(df))
pd.options.mode.chained_assignment=None
to = TabularPandas(df, procs=procs, cat_names=cat_names, cont_names=cont_names, y_names=y_names, y_block=y_block, splits=splits, inplace=True, 
                   reduce_memory=False)
to.show(5)
tab_dls = to.dataloaders(bs=16, val_bs=32)
b = first(tab_dls.train)
test_eq((b[0].shape, b[1].shape, b[2].shape), (torch.Size([16, 7]), torch.Size([16, 3]), torch.Size([16, 1])))

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
9591,Private,HS-grad,Divorced,Adm-clerical,Not-in-family,White,False,49.0,206947.0,9.0,<50k
758,Private,Some-college,Divorced,Tech-support,Not-in-family,White,False,41.0,187802.0,10.0,<50k
4695,Private,Some-college,Never-married,Tech-support,Own-child,White,False,23.0,166371.0,10.0,<50k
4510,Private,Masters,Never-married,Other-service,Not-in-family,White,False,43.0,221550.0,14.0,<50k
32201,Private,Some-college,Never-married,Adm-clerical,Not-in-family,White,False,19.0,220101.0,10.0,<50k


In [ ]:
#hide
out = create_scripts()
beep(out)